In [1]:
# Import packages
import pandas as pd
import numpy as np
from scipy.linalg import inv

# Set pandas number format
np.set_printoptions(precision=6, suppress=True)


Example taken from:

Çetinay, H., Donati, F., Heijungs, R., & Sprecher, B. (2020). Efficient computation of environmentally extended input–output scenario and circular economy modeling. Journal of Industrial Ecology, 24(5), 976–985. https://doi.org/10.1111/jiec.13013


In [2]:
A = np.array(
    [
        [0.0008, 0.0074, 0.0015, 0.0078, 0.0032, 0.0052],
        [0.0025, 0.0014, 0.0091, 0.0069, 0.0091, 0.0063],
        [0.0081, 0.0044, 0.0064, 0.0047, 0.0089, 0.0091],
        [0.0008, 0.0035, 0.0016, 0.0026, 0.0079, 0.0066],
        [0.0053, 0.0048, 0.0057, 0.0057, 0.0093, 0.0039],
        [0.0080, 0.0059, 0.0093, 0.0025, 0.0018, 0.0074],
    ],
)

Y = np.array([0.1188, 0.3800, 0.8128, 0.2440, 0.8844, 0.7126])

bT = np.array([0.8176, 0.6003, 0.849, 0.9223, 0.9476, 0.5270])

change_pct = np.array(
    [
        [0, 0, 0, 0, 0, 0],
        [-0.25, 0, 0, 0, -0.25, 0],
        [0, 0, 0, 0.05, 0, 0],
        [0.35, 0, 0, 0, 0, 0],
        [0.11, 0, 0, 0, 0, 0],
        [0, 0, 0, -0.03, 0.43, 0],
    ],
)

I = np.identity(len(A))


def decompose(A, change):
    MASK = np.any(change, axis=0)
    u = A[:, MASK]
    vT = np.identity(len(A))[MASK]
    return u, vT


$$A⁺ - A = \Delta A = UV^T$$


In [3]:
Adelta = A * change_pct
print(Adelta, "\n")

Astar = Adelta + A
print(Astar)


[[ 0.        0.        0.        0.        0.        0.      ]
 [-0.000625  0.        0.        0.       -0.002275  0.      ]
 [ 0.        0.        0.        0.000235  0.        0.      ]
 [ 0.00028   0.        0.        0.        0.        0.      ]
 [ 0.000583  0.        0.        0.        0.        0.      ]
 [ 0.        0.        0.       -0.000075  0.000774  0.      ]] 

[[0.0008   0.0074   0.0015   0.0078   0.0032   0.0052  ]
 [0.001875 0.0014   0.0091   0.0069   0.006825 0.0063  ]
 [0.0081   0.0044   0.0064   0.004935 0.0089   0.0091  ]
 [0.00108  0.0035   0.0016   0.0026   0.0079   0.0066  ]
 [0.005883 0.0048   0.0057   0.0057   0.0093   0.0039  ]
 [0.008    0.0059   0.0093   0.002425 0.002574 0.0074  ]]


$$u = \sum_i{\Delta a_{ik}e_i} - \sum_j{\Delta a_{jk}e_j}$$
$$v = e_k$$


In [4]:
U, VT = decompose(Adelta, change_pct)

print(U, "\n")
print(VT, "\n")

np.allclose(Adelta, U @ VT)


[[ 0.        0.        0.      ]
 [-0.000625  0.       -0.002275]
 [ 0.        0.000235  0.      ]
 [ 0.00028   0.        0.      ]
 [ 0.000583  0.        0.      ]
 [ 0.       -0.000075  0.000774]] 

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]] 



True

In [5]:
L = inv(I - A)
print(L, "\n")

Lstar = inv(I - Astar)
print(Lstar)


[[1.0009   0.0075   0.001662 0.00792  0.00339  0.005372]
 [0.002689 1.001571 0.009303 0.007064 0.00936  0.00654 ]
 [0.008299 0.004614 1.006644 0.004916 0.009169 0.00937 ]
 [0.000923 0.003608 0.001758 1.00271  0.00806  0.006743]
 [0.005453 0.004964 0.005893 0.005885 1.009558 0.00412 ]
 [0.008173 0.006075 0.009515 0.002688 0.00202  1.00765 ]] 

[[1.0009   0.0075   0.001662 0.00792  0.003377 0.005372]
 [0.002056 1.001555 0.009289 0.007047 0.007063 0.006527]
 [0.008304 0.004614 1.006645 0.005153 0.009167 0.009372]
 [0.001206 0.00361  0.001759 1.002712 0.008058 0.006744]
 [0.00604  0.004968 0.005894 0.00589  1.009552 0.004123]
 [0.008176 0.006079 0.00952  0.002619 0.002793 1.007652]]


$$L^+ = ((I - A) - UV^T)^{-1} = L + LU(I - V^TLU)^{-1}V^TL$$

In [6]:
LU = L @ U
VTL = VT @ L
VTLU = VTL @ U
I_3 = np.identity(len(VT))

res = L + LU @ inv(I_3 - VTLU) @ VTL
print(res, "\n")

np.allclose(Lstar, res)


[[1.0009   0.0075   0.001662 0.00792  0.003377 0.005372]
 [0.002056 1.001555 0.009289 0.007047 0.007063 0.006527]
 [0.008304 0.004614 1.006645 0.005153 0.009167 0.009372]
 [0.001206 0.00361  0.001759 1.002712 0.008058 0.006744]
 [0.00604  0.004968 0.005894 0.00589  1.009552 0.004123]
 [0.008176 0.006079 0.00952  0.002619 0.002793 1.007652]] 



True

$$x^+ = L^+y = Ly + LU(I - V^TLU)^{-1} V^TLy$$
$$\Delta x = LU(I - V^TLU)^{-1} V^TLy$$


In [7]:
xdelta = LU @ inv(I_3 - VTLU) @ VTL @ Y
xdelta


array([-0.000012, -0.002138,  0.000059,  0.000035,  0.00007 ,  0.000674])

$$r^+ = b^T(I - A^+)^{-1} y = b^T(I - A - \Delta A)^{-1} y$$

$$\Delta r = b^T \Delta x = b^T  LU(I - V^TLU)^{-1} V^TLy$$


In [8]:
rdelta = bT @ xdelta
rdelta

-0.0007893352252489563

In [9]:
bT @ Lstar @ Y - bT @ L @ Y

-0.0007893352252494878